In [1]:
import matplotlib.pyplot as plt
from scipy import special
import numpy as np
from lmfit import minimize, Parameters, report_fit, Minimizer
from lmfit.models import GaussianModel, VoigtModel
import seaborn as sns
import time
from scipy import sparse
from scipy.sparse.linalg import spsolve
import pandas as pd

## Non-linear Regression Fitting with a set of Vectors

In [2]:
def Voigt(x,alpha,mu,sigma):
    #z = (x - mu + 1j*sigma)/np.sqrt(2)/sigma
   # w = np.exp(-z**2)*special.erfc(-1j*z)
    #f = alpha*np.real(w)/sigma/np.sqrt(2*np.pi)
    gamma = sigma
    return alpha*np.real(special.wofz(((x-mu) + 1j*gamma)/sigma/np.sqrt(2))) / sigma /np.sqrt(2*np.pi)

def Gaussian(x, alpha, center, sigma):
    return alpha / np.sqrt(2.*np.pi) / sigma * np.exp(-((x-center)/sigma)**2./2.)

In [3]:
def Generate_params(x,Vec,ndata):
    params = Parameters()
    nvec = len(Vec)-1
    if Vec['Alphas'] is None:
        alpha = np.ones((nvec,ndata))*0.1
    else:
        ndata = np.shape(Vec['Alphas'])[1]
        alpha = Vec['Alphas']
    y = np.zeros((ndata,len(x)))
    Min = x.min()
    Max = x.max()
    for i in range(ndata):
        params.add('lin_{}'.format(i), value=0, min=-1, max =1)
        for j in range(nvec):
            npeak = len(Vec['Vec{}'.format(j)]['Mus'])
            if Vec['Vec{}'.format(j)]['Lims'] is None:
                auto = True
            else:    
                auto = False
            for k in range(npeak):
                mu = Vec['Vec{}'.format(j)]['Mus'][k]
                sigma = Vec['Vec{}'.format(j)]['Sigmas'][k]
                if Vec['Vec{}'.format(j)]['Types'][k]:
                    sigma_max_hard = 25.
                    mu_const = 30.
                    sigma_const = 15.
                else: 
                    sigma_max_hard = 6.5
                    mu_const = 5.
                    sigma_const = 5.
                mu_min = mu - mu_const
                mu_max = mu + mu_const
                sigma_min = max(0.1,sigma - sigma_const)
                sigma_max = sigma + sigma_const
                amp_min = -10
                amp_min_k = -10
                if auto == False: 
                    if k in Vec['Vec{}'.format(j)]['Lims']['Peaks']:
                        index = np.where(Vec['Vec{}'.format(j)]['Lims']['Peaks'] == k)[0][0]
                        mu_min = max(mu_min, Vec['Vec{}'.format(j)]['Lims']['Specs'][0,index])
                        mu_max = min(mu_max, Vec['Vec{}'.format(j)]['Lims']['Specs'][1,index])
                        sigma_min = max(0.1, sigma_min, Vec['Vec{}'.format(j)]['Lims']['Specs'][2,index])
                        sigma_max = min(sigma_max, Vec['Vec{}'.format(j)]['Lims']['Specs'][3,index], sigma_max_hard)
                        amp_min = max(-10, Vec['Vec{}'.format(j)]['Lims']['Specs'][4,index])

                if i == 0:
                    amp = Vec['Vec{}'.format(j)]['Amps'][k]*alpha[j,i]
                    print(j,sigma,sigma_min,sigma_max)
                    print(j,mu,mu_min,mu_max)
                    params.add('amp_{}{}{}'.format(i,j,k), value=amp, min=amp_min,  max=10)
                    params.add('cen_{}{}{}'.format(i,j,k), value=mu, min=mu_min,  max=mu_max)
                    params.add('sig_{}{}{}'.format(i,j,k), value=sigma, min=sigma_min,  max=sigma_max)
                else: 
                    if k == 0:
                        if Vec['Vec{}'.format(j)]['Lims'] is not None:
                            if Vec['Vec{}'.format(j)]['Lims']['Pos'] is not None:
                                if Vec['Vec{}'.format(j)]['Lims']['Pos'][k]:
                                    amp_min_k = 0
                        amp = Vec['Vec{}'.format(j)]['Amps'][k]*alpha[j,i]
                        params.add('amp_{}{}{}'.format(i,j,k), value=amp, min=amp_min_k,  max=10)
                    else:
                        a = params['amp_{}{}{}'.format(i,j,0)].value
                        b = params['amp_{}{}{}'.format(0,j,k)].value
                        c = params['amp_{}{}{}'.format(0,j,0)].value
    
                        amp = a*b/c
                        params.add('amp_{}{}{}'.format(i,j,k), min=-10,  max=10,
                                   expr = 'amp_{}{}{}*amp_{}{}{}/amp_{}{}{}'.format(i,j,0,0,j,k,0,j,0))

                    params.add('cen_{}{}{}'.format(i,j,k), min=mu_min,  max=mu_max, expr='cen_{}{}{}'.format(0,j,k))
                    params.add('sig_{}{}{}'.format(i,j,k), min=sigma_min,  max=sigma_max, expr='sig_{}{}{}'.format(0,j,k))
                if Vec['Vec{}'.format(j)]['Types'][k]:
                    y[i] = y[i] + Voigt(x,amp,mu,sigma)
                else: 
                    y[i] = y[i] + Gaussian(x,amp,mu,sigma)
                #print('{}{}{}\t{}\t{}\t{}'.format(i,j,k,amp,mu,sigma))
    return params

In [5]:
def Generate_data(params,x,Vec,ndata):
    nvec = len(Vec)-1
    if Vec['Alphas'] is None:
        alpha = np.ones((nvec,ndata))
    else:
        ndata = np.shape(Vec['Alphas'])[1]
        alpha = Vec['Alphas']
    y = np.zeros((ndata,len(x)))
       
    for i in range(ndata):
        y[i] = y[i] + params['lin_{}'.format(i)].value
        for j in range(nvec):
            npeak = len(Vec['Vec{}'.format(j)]['Mus'])
            for k in range(npeak):
                amp = params['amp_{}{}{}'.format(i,j,k)].value
                mu = params['cen_{}{}{}'.format(i,j,k)].value
                sigma = params['sig_{}{}{}'.format(i,j,k)].value

                if Vec['Vec{}'.format(j)]['Types'][k]:
                    y[i] = y[i] + Voigt(x,amp,mu,sigma)
                else: 
                    y[i] = y[i] + Gaussian(x,amp,mu,sigma)
    return y

In [15]:
def Plot_data(params,x,y,Vec,time,Dif):
    print('Plotting.....')
    nvec = len(Vec)-1
    ndata = len(y)
    yhat = np.zeros((ndata,len(x)))
    ## ax1 is for plotting the vectors
    fig,ax1 = plt.subplots(1,1,figsize = (4,3),dpi=100)
    ## axes is for plotting the fits
    if ndata<4:
        ncol = ndata
        nrow = 1
    else:
        ncol = 4
        if ndata%4 == 0:
            nrow = int((ndata - ndata%4)/4)
        else:
            nrow = int((ndata - ndata%4)/4)+1
    fig,axes = plt.subplots(nrow,ncol,figsize=(3*ncol,3*nrow),dpi=150)
    
    c = ['c','g','k','m','y']
    amps = np.zeros((ndata,nvec))
    for i in range(ndata):
        yhat[i]= yhat[i] + params['lin_{}'.format(i)].value
        if nrow == 1:
            Obj=axes[i%ncol]
        else:
            Obj=axes[int((i - i%ncol)/ncol), i%ncol]
        for j in range(nvec):
            npeak = len(Vec['Vec{}'.format(j)]['Mus'])
            vec = np.zeros(len(x))
            amp_temp = 0
            for k in range(npeak):
                amp = params['amp_{}{}{}'.format(i,j,k)].value
                mu = params['cen_{}{}{}'.format(i,j,k)].value
                sigma = params['sig_{}{}{}'.format(i,j,k)].value
                amp_temp = amp_temp + amp
                if Vec['Vec{}'.format(j)]['Types'][k]:
                    yhat[i] = yhat[i] + Voigt(x,amp,mu,sigma)
                    vec = vec + Voigt(x,amp,mu,sigma)
                    #axes[int((i - i%4)/4), i%4].plot(x,Voigt(x,amp,mu,sigma))
                else: 
                    yhat[i] = yhat[i] + Gaussian(x,amp,mu,sigma)
                    vec = vec + Gaussian(x,amp,mu,sigma)    
                    #axes[int((i - i%4)/4), i%4].plot(x,Gaussian(x,amp,mu,sigma))
            amps[i,j] = amp_temp
            Obj.plot(x,vec)
            if i == 0:
                ax1.plot(x,vec/amp_temp,c=c[j],label='Vec{}'.format(j))

        Obj.plot(x,yhat[i],'r--',label='fit')
        Obj.plot(x,y[i],'b',label='data')
        Obj.set_xlabel('wavenumber (cm$^{-1}$)')
        Obj.set_ylabel('Absorbance (a.u.)')
        Obj.legend()
        Obj.invert_xaxis()
        Obj.set_title('{} min'.format(time[i]))
        
    ax1.legend()
    ax1.set_xlabel('wavenumber (cm$^{-1}$)')
    ax1.set_ylabel('Absorbance (a.u.)')
    ax1.set_title('Vector basis')
    ax1.invert_xaxis()
    plt.tight_layout()
    if Dif:
        amps = amps.cumsum(axis=0)
        
    fig, axes1 = plt.subplots(1,nvec,figsize=(nvec*3,3),dpi=100)
    if nvec == 1:
        axes1.scatter(time,amps[:,j])
        axes1.set_xlabel('Time (min)')
        axes1.set_ylabel('Area (a.u.)')
        axes1.set_ylim((min(amps[:,j].min(),0)-amps[:,j].max()*0.1,amps[:,j].max()*1.1))    
    else:
        for j in range(nvec):
            axes1[j].scatter(time,amps[:,j])
            axes1[j].set_xlabel('time (min)')
            axes1[j].set_ylabel('Area (a.u.)')
            axes1[j].set_title('Vec{}'.format(j))
            axes1[j].set_ylim((min(amps[:,j].min(),0)-amps[:,j].max()*0.1,amps[:,j].max()*1.1))
    plt.tight_layout()
    
    fig, ax2 = plt.subplots(1,1,figsize=(4,3),dpi=100)
    for j in range(nvec):
        Maxj = amps[:,j].max()
        ax2.scatter(time,amps[:,j]/Maxj,label='Vec{}'.format(j))
    ax2.set_xlabel('Time (min)')
    ax2.set_ylabel('Normalized area (a.u.)')
    ax2.set_ylim((-0.1,1.1))
    ax2.legend()
    plt.tight_layout()

    return None

In [7]:
def loss_func(params, x, y, Vec):
    """ calculate total residual for fits to several data sets held
    in a 2-D array, and modeled by Gaussian functions"""
    ndata, nx = y.shape
    resid = 0.0*y
    # make residual per data set
    model = Generate_data(params,x,Vec,ndata)
    resid = np.transpose((y-model))
    # now flatten this to a 1D array, as minimize() needs
    return resid.flatten()

In [ ]:
def Plot_com_data(Param_com, Vec, Time, Dif):
    nPara_set = len(Param_com['Params'])
    nVec = len(Vec['Peaks'])
    data = np.zeros((nVec,len(Time)))
    for l in range(nPara_set):
        ndata = Param_com['Lens'][l]
        for i in range(ndata):
            if l == 0:
                T_ind = i
            else: 
                T_ind = i + np.cumsum(Param_com['Lens'])[l-1]
            for j in range(nVec):
                amp_tem = 0 
                nPeak = len(Vec['Peaks'][j])
                for k in range(nPeak):
                    amp_tem = amp_tem + Param_com['Params'][l]['amp_{}{}{}'.format(i,j,k)]
                
                data[j,T_ind] = amp_tem
    data = np.cumsum(data, axis=1)
    fig, axes = plt.subplots(1,nVec,figsize=(nVec*3, 3), dpi = 150)
    for i in range(nVec):
        axes[i].scatter(Time,data[i])
        axes[i].set_xlabel('Time (min)')
        axes[i].set_ylabel('Peak area (a.u.)')
        axes[i].set_title('Vec{}'.format(i))
    plt.tight_layout()
    return data

In [ ]:
def Print_var(Param,Name,Vec,Assignment=None):
    print('Species\t\tType\t\tPeak (cm-1)\tVariance (cm-1)\t')
    nVec = len(Name)
    data = np.zeros((nVec,len(Time)))
    for j in range(nVec):
        print(Name[j], end='\t')
        nPeak = len(Vec['Peaks'][j])
        for k in range(nPeak):
            if k != 0:
                print('\t',end='\t')
            if Vec['Types'][j][k]:
                print('Voigt', end='\t\t')
            else:
                print('Gaussian', end='\t')
            mu = Param['cen_{}{}{}'.format(0,j,k)].value
            sigma = Param['sig_{}{}{}'.format(0,j,k)].value

            print('{:.1f}\t\t{:.1f}'.format(mu,sigma))
    return None

In [8]:
def get_first_nbr_from_str(input_str):
    '''
    :param input_str: strings that contains digit and words
    :return: the number extracted from the input_str
    demo:
    'ab324.23.123xyz': 324.23
    '.5abc44': 0.5
    '''
    if not input_str and not isinstance(input_str, str):
        return 0
    out_number = ''
    for ele in input_str:
        if (ele == '.' and '.' not in out_number) or ele.isdigit():
            out_number += ele
        elif out_number:
            break
    return float(out_number)

In [9]:
def baseline_als(y, lam, p, niter=10):
    L = len(y)
    D = sparse.diags([1,-2,1],[0,-1,-2], shape=(L,L-2))
    w = np.ones(L)
    for i in range(niter):
        W = sparse.spdiags(w, 0, L, L)
        Z = W + lam * D.dot(D.transpose())
        z = spsolve(Z, w*y)
        w = p * (y > z) + (1-p) * (y < z)
    return z

## Finding a good initial guess

By implementing the lmfit package, we can performance the optimzation of our loss function by minimizing the least square of the difference between the model and actual data. The 'leastsq' method from the lmfit package uses the 'Levenberg-Marquardt' algorithm for the minization. While often criticized, including the fact it finds
a local minima, this approach has some distinct advantages. These include being fast, and well-behaved for most
curve-fitting needs, and making it easy to estimate uncertainties for and correlations between pairs of fit variables. Still the disadvantage of the 'Levenberg-Marquardt' algorithm which is its looking for local minima should be addressed. Hence, the first step in our modeling methodology is finding a reasonable initial guess so that further optimization would have better performances. 

In [10]:
def Generate_params_IG(x,Vec,ndata):
    params = Parameters()
    npeak = len( Vec['Mus'])
    alpha = np.ones((npeak,ndata))*0.05

    Min = x.min()
    Max = x.max()
    if Vec['Lims'] is None:
        amp_min = -1
        Mus_min_bool = False
    else:
        if Vec['Lims']['Positive'] is None:
            amp_min = -1
        else:
            if Vec['Lims']['Positive']:
                amp_min = 0
            else:
                amp_min = -1
        if Vec['Lims']['Mus_min'] is None:
            Mus_min_bool = False
        else:
            Mus_min_bool = True
        
    for i in range(ndata):
        params.add('lin_{}'.format(i), value=0, min=-1, max =1)
        for j in range(npeak):
            mu = Vec['Mus'][j]
            sigma = Vec['Sigmas'][j]
            amp = alpha[j,i]
            if Mus_min_bool:
                mu_min = Vec['Lims']['Mus_min'][j]
            else:
                mu_min = mu*0.9
            if i ==0:
                params.add('amp_{}{}'.format(i,j), value=amp, min=amp_min,  max=1)
                params.add('cen_{}{}'.format(i,j), value=mu, min=mu_min,  max=1.1*mu)
                params.add('sig_{}{}'.format(i,j), value=sigma, min=0.7*sigma,  max=1.3*sigma)
            else:
                params.add('amp_{}{}'.format(i,j), value=amp, min=amp_min,  max=1)
                params.add('cen_{}{}'.format(i,j), min=Min,  max=Max, expr='cen_{}{}'.format(0,j))
                params.add('sig_{}{}'.format(i,j), min=0.1,  max=10, expr='sig_{}{}'.format(0,j))
    return params

In [11]:
def Generate_data_IG(params,x,Vec,ndata):
    npeak = len(Vec['Mus'])
    alpha = np.ones((npeak,ndata))
    y = np.zeros((ndata,len(x)))
    
    for i in range(ndata):
        y[i] = y[i] + params['lin_{}'.format(i)].value
        for j in range(npeak):
            amp = params['amp_{}{}'.format(i,j)].value
            mu = params['cen_{}{}'.format(i,j)].value
            sigma = params['sig_{}{}'.format(i,j)].value
            if Vec['Types'][j]:
                y[i] = y[i] + Voigt(x,amp,mu,sigma)
            else: 
                y[i] = y[i] + Gaussian(x,amp,mu,sigma)
    return y

In [12]:
def loss_func_IG(params, x, y, Vec):
    """ calculate total residual for fits to several data sets held
    in a 2-D array, and modeled by Gaussian functions"""
    ndata, nx = y.shape
    resid = 0.0*y
    # make residual per data set
    model = Generate_data_IG(params,x,Vec,ndata)
    resid = y-model
    # now flatten this to a 1D array, as minimize() needs
    return resid.flatten()

In [13]:
def Plot_data_IG(params,x,y,Vec,time,Dif):
    npeak = len(Vec['Mus'])
    ndata = len(y)
    yhat = np.zeros((ndata,len(x)))
    if ndata<4:
        ncol = ndata
        nrow = 1
    else:
        ncol = 4
        if ndata%4 == 0:
            nrow = int((ndata - ndata%4)/4)
        else:
            nrow = int((ndata - ndata%4)/4)+1
    fig,axes = plt.subplots(nrow,ncol,figsize=(3*ncol,3*nrow),dpi=150)
    amps = np.zeros((ndata,npeak))
    c = ['c','g','k','m','y']
    for i in range(ndata):
        if nrow == 1:
            Obj=axes[i%ncol]
        else:
            Obj=axes[int((i - i%ncol)/ncol), i%ncol]
        yhat[i]= yhat[i] + params['lin_{}'.format(i)].value
        for j in range(npeak):
            amp = params['amp_{}{}'.format(i,j)].value
            mu = params['cen_{}{}'.format(i,j)].value
            sigma = params['sig_{}{}'.format(i,j)].value
            if Vec['Types'][j]:
                yhat[i] = yhat[i] + Voigt(x,amp,mu,sigma)
                Obj.plot(x,Voigt(x,amp,mu,sigma))
            else: 
                yhat[i] = yhat[i] + Gaussian(x,amp,mu,sigma)
                Obj.plot(x,Gaussian(x,amp,mu,sigma))
            amps[i,j] = amp
               
        Obj.plot(x,yhat[i],'r--',label='fit')
        Obj.plot(x,y[i],'b',label='data')
        Obj.set_xlabel('wavenumber (cm$^{-1}$)')
        Obj.set_ylabel('Absorbance (a.u.)')
        Obj.legend()
        Obj.invert_xaxis()
        Obj.set_title('{} min'.format(time[i]))
        
    plt.tight_layout()
    if Dif:
        amps = amps.cumsum(axis=0)
    fig, axes1 = plt.subplots(1,npeak,figsize=(npeak*3.5,3),dpi=150)
    for j in range(npeak):
        axes1[j].scatter(time,amps[:,j])
        axes1[j].set_xlabel('time (min)')
        axes1[j].set_ylabel('Area (a.u.)')
        axes1[j].set_ylim((min(amps[:,j].min(),0)-amps[:,j].max()*0.1,amps[:,j].max()*1.1))
        axes1[j].set_title('{} cm-1'.format(params['cen_0{}'.format(j)].value))
    plt.tight_layout()
    
    fig, ax2 = plt.subplots(1,1,figsize=(4,3),dpi=100)
    for j in range(npeak):
        Maxj = amps[:,j].max()
        Norm_amps = amps[:,j]/Maxj
        ax2.scatter(time,Norm_amps,label = '{:.1f}'.format(params['cen_0{}'.format(j)].value))
        ax2.set_xlabel('time (min)')
        ax2.set_ylabel('Normalized area (a.u.)')
        ax2.set_ylim((-0.1,1.1))
    plt.tight_layout()
    #Correlation plot
    PeakPos = []
    for j in range(npeak):
        PeakPos.append(int(params['cen_0{}'.format(j)].value))
    df = pd.DataFrame(amps,columns = PeakPos)
    corr = df.corr()
    
    f, ax = plt.subplots(figsize=(5, 4),dpi=150)

    sns.heatmap(corr, 
                square=True, linewidths=.5, cbar_kws={"shrink": .5})
    plt.show()
    results = {'Peak Pos': PeakPos,'Amps': amps}
    return results

In [14]:
def Generate_Vec_IG(params, Vec_IG, ndata):
    nVec = len(Vec_IG['Peaks'])
    Vec = {}
    Alphas = np.zeros((nVec,ndata))
    for i in range(nVec):
        Vec_tem = {}
        npeak = len(Vec_IG['Peaks'][i])
        Mus = []
        Sigmas = []
        Amps = []
        Types = Vec_IG['Types'][i]
        for j in range(npeak):
            k = Vec_IG['Peaks'][i][j]
            Mus.append(params['cen_{}{}'.format(0,k)].value)
            Sigmas.append(params['sig_{}{}'.format(0,k)].value)
            Amps_l = []
            for l in range(ndata):
                Amps_l.append(params['amp_{}{}'.format(l,k)].value)
            Amps_l = np.array(Amps_l)
            Amps.append(Amps_l.mean())
        Mus = np.array(Mus)
        Sigmas = np.array(Sigmas)
        Amps = np.array(Amps)
        Vec_tem['Mus'] =  Mus
        Vec_tem['Sigmas'] = Sigmas
        Vec_tem['Amps'] = Amps
        Vec_tem['Types'] = Types
        Vec['Vec{}'.format(i)] = Vec_tem
        Vec['Vec{}'.format(i)]['Lims'] = None
    for i in range(ndata):
        for j in range(nVec):
            npeak = len(Vec_IG['Peaks'][j])
            amp_tem = 0
            for k in range(npeak):
                l = Vec_IG['Peaks'][j][k]
                amp_tem = amp_tem + params['amp_{}{}'.format(i,l)].value
            Alphas[j,i] = amp_tem
            
    Vec['Alphas'] = Alphas
    return Vec